In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [ ]:
team_dict, team_list  = get_teams()

In [ ]:
from nba_api.stats.endpoints import scoreboard

In [ ]:
dfa = []
for do in range(60):
    stats = scoreboard.Scoreboard(
        game_date = "2024-02-25",
        league_id="00",
        day_offset=do,
    )
    df1 = stats.get_data_frames()[0]
    dfa.append(df1)
    time.sleep(0.6)

In [ ]:
df = pd.concat(dfa)

In [ ]:
df = df[~df.duplicated(keep="last")].reset_index(drop=True)

In [ ]:
df["Home_Team"] = df["HOME_TEAM_ID"].map(team_dict)
df["Away_Team"] = df["VISITOR_TEAM_ID"].map(team_dict)

In [ ]:
dfh = pd.DataFrame( df["Home_Team"].value_counts()).reset_index()
dfh = dfh.rename(columns={"Home_Team":"Team","count":"Home"})
dfv = pd.DataFrame( df["Away_Team"].value_counts()).reset_index()
dfv = dfv.rename(columns={"Away_Team":"Team","count":"Away"})

In [ ]:
df2 = pd.merge(dfh,dfv)

In [ ]:
df2["Diff"] = df2["Home"] - df2["Away"]

In [ ]:
df2 = df2.sort_values("Diff",ascending=False)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df2
df <- df2
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Remaining Home - Away Games**"),
    ) %>%
    gt_plt_bar(column = Diff, keep_column = TRUE, color = "red") %>%
    data_color(columns = c(Diff), palette = "PiYG") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/home_away.png",size=5) 